# Central Bank Policy Rates - BIS

## Python set-up

In [1]:
# system imports
import datetime as dt
from pathlib import Path

In [2]:
# analytic imports
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import pandasdmx as sdmx
import pycountry

In [3]:
# local imports
from plotting import (
    finalise_plot,
    set_chart_dir,
    plot_series_highlighted,
    line_plot,
)

In [4]:
# plotting set-up
SOURCE = "Source: BIS policy rates"
LFOOTER = "Monthly data. There are lags in BIS data reporting. "
plt.style.use("fivethirtyeight")
CHART_DIR = "./CHARTS/BIS/"
Path(CHART_DIR).mkdir(parents=True, exist_ok=True)
set_chart_dir(CHART_DIR)
for filename in Path(CHART_DIR).glob("*.png"):
    filename.unlink()
SHOW = False

## Data capture

In [5]:
def get_bis_monthly():
    """Get a table of monthly policy rates from the BIS SDMX web service."""

    def country_code_to_name(country_code: str) -> str:
        """Convert 2-digit country codes to country names."""

        try:
            country = pycountry.countries.get(alpha_2=country_code)
            return country.name
        except AttributeError:
            if country_code == "XM":
                return "Euro Area"
            return country_code

    def get_bis(
        conn: sdmx.api.Request, datastr: str, freq: str, start: str
    ) -> pd.DataFrame:
        """Get data from an SDMX web service (conn in the arguments)."""

        datasets = conn.dataflow()
        dataset = [
            x for x in datasets.dataflow.values() if datastr in str(x.name).lower()
        ][0]
        data = (
            conn.data(
                resource_id=dataset.id,
                key={"FREQ": freq},
                params={"startPeriod": start},
            )
            .to_pandas()
            .unstack()
            .T
        )
        data.columns = data.columns.droplevel()
        return data

    # Create a connection to the BIS SDMX web service
    conn = sdmx.Request("BIS")

    # Get monthly policy rate data all states
    since = "1992-01"
    monthly_policy_rates = get_bis(conn, "policy rates monthly", "M", since)
    monthly_policy_rates.index = pd.PeriodIndex(monthly_policy_rates.index, freq="M")
    print(f"Latest monthly data: {monthly_policy_rates.index[-1]}")

    # get recent daily policy rates data all states, and convert to monthly
    recent = 90  # days
    start = (dt.datetime.now() - dt.timedelta(days=recent)).strftime("%Y-%m-%d")
    daily_policy_rates = get_bis(conn, "policy rates daily", "D", start)
    daily_policy_rates.index = pd.to_datetime(daily_policy_rates.index)
    print(f"Latest daily data: {daily_policy_rates.index[-1]}")

    recent1 = daily_policy_rates.to_period("M")
    recent2 = recent1.groupby(by=[recent1.index.year, recent1.index.month]).ffill()
    recent3 = recent2.groupby(by=[recent2.index.year, recent2.index.month]).last()  # skipna=?
    recent3.index.set_names(["year", "month"], inplace=True)
    recent3.index = pd.PeriodIndex([f"{y}-{m}" for y, m in recent3.index], freq="M")

    # amalgamate the monthly and daily dataframes, as monthly data
    columns = monthly_policy_rates.columns.intersection(recent3.columns)
    index = monthly_policy_rates.index.union(recent3.index)
    monthly_policy_rates = monthly_policy_rates.reindex(index=index)
    recent3 = recent3.reindex(index)
    monthly_policy_rates = monthly_policy_rates[columns]
    monthly_policy_rates = monthly_policy_rates.where(
        monthly_policy_rates.notna(), other=recent3
    )
    monthly_policy_rates.columns = [
        country_code_to_name(c) for c in monthly_policy_rates.columns
    ]

    return monthly_policy_rates


policy_rates = get_bis_monthly()

Latest monthly data: 2024-03
Latest daily data: 2024-04-08 00:00:00


In [6]:
print(policy_rates.columns)

Index(['Argentina', 'Australia', 'Brazil', 'Canada', 'Switzerland', 'Chile',
       'China', 'Colombia', 'Czechia', 'Denmark', 'United Kingdom',
       'Hong Kong', 'Hungary', 'Indonesia', 'Israel', 'India', 'Iceland',
       'Japan', 'Korea, Republic of', 'Morocco', 'North Macedonia', 'Mexico',
       'Malaysia', 'Norway', 'New Zealand', 'Peru', 'Philippines', 'Poland',
       'Romania', 'Serbia', 'Russian Federation', 'Saudi Arabia', 'Sweden',
       'Thailand', 'Turkey', 'United States', 'Euro Area', 'South Africa'],
      dtype='object')


## Plotting

In [7]:
def plot_policy_rates(pr: pd.DataFrame):
    """Plot policy rates for all countries in the dataframe."""

    for col in pr.columns:
        series = pr[col]
        ax = plot_series_highlighted(series)
        finalise_plot(
            ax,
            y0=True,
            title=f"Central Bank Policy Rate: {col}",
            rfooter=SOURCE,
            lfooter=LFOOTER,
            show=SHOW,
        )


plot_policy_rates(policy_rates)

In [8]:
def plot_by_group(pr: pd.DataFrame) -> None:
    """Plot policy rates by group."""

    chart_groups = (
        (
            "Australia",
            "Canada",
            "New Zealand",
            "United Kingdom",
            "United States",
        ),
        ("Brazil", "Russian Federation", "India", "China", "South Africa"),
        (
            "Denmark",
            "Euro Area",
            "Norway",
            "Switzerland",
            "Sweden",
            "Iceland",
        ),
        (
            "Czechia",
            "Hungary",
            "North Macedonia",
            "Poland",
            "Romania",
            "Serbia",
        ),
        (
            "Thailand",
            "Malaysia",
            "Philippines",
            "Indonesia",
            "Korea, Republic of",
            "Japan",
        ),
        (
            "Chile",
            "Colombia",
            "Mexico",
            "Peru",
        ),
        (
            "Saudi Arabia",
            "Hong Kong",
            "Israel",
            "Morocco",
        ),
        ("Argentina", "Turkey"),
    )
    since = pd.Period("2021-01", freq="M")
    for number, group in enumerate(chart_groups):
        group_data = pr.loc[since:, group]
        cmap = mpl.colormaps["Set1"]
        line_colors = cmap(np.linspace(0, 1, len(group)))
        line_colors_hex = [mcolors.rgb2hex(color) for color in line_colors]
        ax = group_data.plot(drawstyle="steps-post", color=line_colors_hex, lw=2.5)
        finalise_plot(
            ax,
            y0=True,
            title="Central Bank Policy Rates",
            ylabel="Policy Rate (%)",
            rfooter=SOURCE,
            lfooter=LFOOTER,
            tag=f"group-{number}",
            show=SHOW,
        )


plot_by_group(policy_rates)

In [9]:
def plot_range_since_covid(pr: pd.DataFrame) -> None:
    """Plot Central Bank Policy Rates since COVID-19."""

    # get data since COVID-19
    since = pd.Period("2021-01", freq="M")
    def open(x: pd.Series) -> float: return x.dropna().iloc[0]
    def close(x: pd.Series) -> float: return x.dropna().iloc[-1]
    ohlc = (
        pr
        .loc[since:]
        .dropna(how='all', axis=1)
        .agg([ open, "max", "min", close, ], axis=0)
        .T
    )

    # exclude countries with rates above a cutoff
    cutoff = 20  # per cent - exclude countries with rates above this
    excluded = ohlc[ohlc["max"] >= cutoff].index
    ohlc = ohlc[~ohlc.index.isin(excluded)].sort_values(by="close")

    # prepare to plot the data
    good, bad = "darkblue", "darkorange"
    colors = [
        good if open > close else bad
        for open, close in zip(ohlc.open, ohlc.close)
    ]
    minimum = ohlc["min"].min()
    maximum = ohlc["max"].max()
    adjustment = (maximum - minimum) * 0.025
    limits = (minimum - adjustment, maximum + adjustment)

    # plot the data
    _fig, ax = plt.subplots()
    ax.bar(
        ohlc.index, 
        height=ohlc["max"] - ohlc["min"],
        bottom=ohlc["min"],
        color=colors,
        linewidth=1.0,
        edgecolor="black",
        label=f"Range of policy rates since {since}",
        alpha=0.15,
    )
    ax.plot(
        ohlc.index,
        ohlc.open,
        marker="<",
        linestyle="None",
        label=f"Policy rate at {since}",
        color=good,
        markersize=5,
    )
    ax.plot(
        ohlc.index,
        ohlc.close,
        marker=">",
        linestyle="None",
        label=f"Latest policy rate",
        color=bad,
        markersize=5,
    )
    ax.set_xticklabels(ohlc.index, rotation=90)
    ax.tick_params(axis="both", which="major", labelsize="x-small")
    finalise_plot(
        ax,
        y0=True,
        ylim=limits,
        title=f"Central Bank Policy Rates since {since}",
        ylabel="Policy Rate (%)",
        legend={"loc": "upper left", "fontsize": "x-small",},
        rfooter=SOURCE,
        lfooter=f"{LFOOTER} Excluded: {', '.join(excluded.to_list())}",
        show=SHOW,
    )


plot_range_since_covid(policy_rates)

/var/folders/96/8bhlz_x975z93glbxq_2_yqr0000gn/T/ipykernel_86556/2906174114.py:62: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(ohlc.index, rotation=90)


In [10]:
# Australia vs US
def plot_aus_us(pr: pd.DataFrame) -> None:
    """Plot the policy rates of Australia and the United States."""

    plot = pr[["United States", "Australia"]]
    line_plot(
        plot,
        title="Central Bank Policy Rates: Australia vs United States",
        ylabel="Policy Rate (%)",
        width=[1, 2],
        drawstyle="steps-post",
        y0=True,
        rfooter=SOURCE,
        lfooter=LFOOTER,
        show=False,
    )


plot_aus_us(policy_rates)

## The End

In [11]:
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Fri Apr 12 2024 21:36:48

Python implementation: CPython
Python version       : 3.12.2
IPython version      : 8.22.2

pandasdmx : 1.8.1
numpy     : 1.26.4
pycountry : 22.3.5
matplotlib: 3.8.4
pandas    : 2.2.1

Watermark: 2.4.3

